In [1]:
import datasets
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import ast
import os
import tiktoken
from src.utils.git_utils import get_changed_files_between_commits, get_repo_content_on_commit, parse_changed_files_and_lines_from_diff

# Bug localization datasets metrics

In [31]:
df = pd.read_csv('/mnt/data/shared-data/lca/bug_localization_data/metrics.csv')
df[:5]

,id,text_id,repo_owner,repo_name,issue_url,pull_url,comment_url,links_count,issue_title,link_keyword,...,repo_files_without_tests_count,changed_symbols_count,changed_tokens_count,changed_lines_count,changed_files_without_tests_count,issue_symbols_count,issue_tokens_count,issue_lines_count,issue_links_count,issue_code_blocks_count
0,8543,thealgorithms/python/295/289,thealgorithms,python,https://github.com/TheAlgorithms/Python/issues...,https://github.com/TheAlgorithms/Python/pull/295,https://github.com/TheAlgorithms/Python/pull/295,1,ProjectEuler -- Problem 1 -- solv2.py -- Error,fixes,...,195,357,159,12,1,191,57.0,2,1,3
1,5531,electron/electron/8668/8555,electron,electron,https://github.com/electron/electron/issues/8555,https://github.com/electron/electron/pull/8668,https://github.com/electron/electron/pull/8668,1,Mac app store build uses non-public APIs,close,...,34,95,51,2,1,539,128.0,15,0,0
2,5532,electron/electron/8640/8608,electron,electron,https://github.com/electron/electron/issues/8608,https://github.com/electron/electron/pull/8640,https://github.com/electron/electron/pull/8640,1,Debug symbols not generated for Release build,close,...,34,418,140,8,2,154,36.0,1,0,0
3,5530,electron/electron/11103/11101,electron,electron,https://github.com/electron/electron/issues/11101,https://github.com/electron/electron/pull/11103,https://github.com/electron/electron/pull/11103,1,process.versions does not include new version ...,closes,...,1307,1064,244,27,3,261,71.0,14,0,0
4,6847,keras-team/keras/18352/15282,keras-team,keras,https://github.com/keras-team/keras/issues/15282,https://github.com/keras-team/keras/pull/18352,https://github.com/keras-team/keras/pull/18352,1,MobileNetV3 models can't infer the static shape,solve,...,677,100,39,2,1,5225,1229.0,103,3,1


# Repo metrics analysis

In [5]:
df[['repo_symbols_count', 'repo_tokens_count', 'repo_lines_count', 'repo_files_without_tests_count']].describe(percentiles=[.01, .25, .5, .75, .99])

,repo_symbols_count,repo_tokens_count,repo_lines_count,repo_files_without_tests_count
count,1.019500e+04,1.015300e+04,1.019500e+04,10195.000000
mean,6.602220e+06,1.754613e+06,1.486032e+05,1085.957136
std,2.091108e+07,7.355560e+06,3.673932e+05,2057.825737
min,3.210000e+02,7.800000e+01,9.000000e+00,1.000000
1%,8.511722e+04,1.882836e+04,2.582520e+03,15.000000
25%,6.510080e+05,1.449600e+05,1.795200e+04,117.000000
50%,2.115322e+06,4.995300e+05,5.337000e+04,333.000000
75%,5.450736e+06,1.261706e+06,1.448790e+05,1114.500000
99%,5.325579e+07,1.559571e+07,9.565990e+05,7854.540000
max,7.883723e+08,2.256497e+08,8.688752e+06,33644.000000


In [6]:
# Remove none values repo_tokens_count
len(df[df['repo_tokens_count'].isnull()])

42

# Diff metrics analysis

In [8]:
df[['changed_symbols_count', 'changed_tokens_count', 'changed_lines_count', 'changed_files_count', 'changed_files_without_tests_count']].describe(percentiles=[.01, .25, .5, .75, .99])

,changed_symbols_count,changed_tokens_count,changed_lines_count,changed_files_count,changed_files_without_tests_count
count,1.019500e+04,1.019500e+04,10195.000000,10195.000000,10195.000000
mean,4.323743e+03,1.382341e+03,72.649338,4.829917,3.996763
std,6.390877e+04,2.849731e+04,1437.670638,108.235589,104.134846
min,0.000000e+00,0.000000e+00,0.000000,1.000000,1.000000
1%,3.800000e+01,8.000000e+00,1.000000,1.000000,1.000000
25%,2.940000e+02,6.300000e+01,6.000000,1.000000,1.000000
50%,7.590000e+02,1.630000e+02,15.000000,2.000000,1.000000
75%,2.044500e+03,4.280000e+02,42.000000,3.000000,3.000000
99%,3.402712e+04,7.719560e+03,594.420000,22.000000,17.000000
max,4.513281e+06,1.649023e+06,139716.000000,10769.000000,10385.000000


In [9]:
ind = df['changed_files_count'].idxmax()
print(df.loc[ind]['issue_body'])

Confirm by changing [ ] to [x] below to ensure that it's a bug:
- [x] I've gone though the [API reference](https://docs.aws.amazon.com/sdk-for-go/v2/api/)
- [x] I've checked [AWS Forums](https://forums.aws.amazon.com) and [StackOverflow](https://stackoverflow.com/questions/tagged/aws-sdk-go) for answers
- [x] I've searched for [previous similar issues](https://github.com/aws/aws-sdk-go-v2/issues) and didn't find any solution
  
**Describe the bug**
When describing an InstanceType like "t3a.large" or "t3a.xlarge" in returned 'InstanceTypeInfo' information about vCPUs is empty - `VCpuInfo` is nil.

In comparison `MemoryInfo` is fine and has all the details.

aws-sdk-go v1 does not have this issue and works as expected.

**Version of AWS SDK for Go?**
v0.29.0

**Version of Go (`go version`)?**
v1.15.2

**To Reproduce (observed behavior)**

	reqInput := &ec2.DescribeInstanceTypesInput{
		InstanceTypes: []types.InstanceType{types.InstanceType("t3a.large")},
	}

	info, err := p.ec2.DescribeI

In [10]:
# Remove none values in changed_tokens_count
print("Failed to count tokens: {} rows to be deleted".format(
    len(df[df['changed_tokens_count'].isnull()]))
)

Failed to count tokens: 0 rows to be deleted


In [11]:
# Remove outliers with lots of changed files
print("Too much changed files: {} rows to be deleted".format(
    len(df[df['changed_files_count'] > 22]))
)

Too much changed files: 100 rows to be deleted


In [12]:
# Remove outliers with lots of changed lines
print("Too much changed files: {} rows to be deleted".format(
    len(df[df['changed_lines_count'] > 594]))
)

Too much changed files: 102 rows to be deleted


# Issue description analysis

In [14]:
df[['issue_symbols_count', 'issue_tokens_count', 'issue_lines_count', 'issue_links_count', 'issue_code_blocks_count']].describe(percentiles=[.01, .25, .5, .75, .99])

,issue_symbols_count,issue_tokens_count,issue_lines_count,issue_links_count,issue_code_blocks_count
count,10195.000000,10194.000000,10195.000000,10195.000000,10195.000000
mean,1949.124865,521.842456,36.545463,0.945562,0.995586
std,4385.789636,1326.001422,59.989556,7.531692,1.321217
min,12.000000,2.000000,1.000000,0.000000,0.000000
1%,60.000000,13.000000,1.000000,0.000000,0.000000
25%,409.000000,99.000000,8.000000,0.000000,0.000000
50%,895.000000,227.000000,22.000000,0.000000,1.000000
75%,1963.500000,525.000000,45.000000,1.000000,2.000000
99%,16651.920000,4491.490000,232.000000,6.000000,6.000000
max,132846.000000,51592.000000,2402.000000,601.000000,31.000000


In [15]:
ind = df['issue_tokens_count'].idxmax()
print(df.loc[ind]['issue_body'])

**Describe the bug**
After searching for (older) email, used Reply and Reply All before selecting "Download Complete Message" resulted in error.  First time returns K9 to the inbox list.  After repeated attempts, K9 will FC. 

**To Reproduce**
Steps to reproduce the behavior:
1. From inbox or unified inbox, searched for older message
2.  found message with "Download Complete Message" shown at bottom (doesn't matter if attachment or not)
3.  Did NOT click on "Download Complete Message" but instead selected Reply (and Reply all - same result)
4.  After clicking Reply or Reply all, K9 reverted to inbox list of email (did not show new email response)
5.  With (2 or 3) repeated attempts, K9 then crashed/FC

**Expected behavior**
Reply or Reply All yields a new message that can edited/typed and sent.

**Screenshots**
n/a

**Environment (please complete the following information):**
 - K-9 Mail version: 5.734
 - Android version: 10  (Samsung ver FFUE1)
 - Device: Galaxy S9 SM-G960F 
 - Accoun

In [16]:
# Remove outliers with lots of changed files
print("Too large issue description: {} rows to be deleted".format(
    len(df[df['issue_tokens_count'] > 4491]))
)

Too large issue description: 102 rows to be deleted


In [17]:
ind = df['issue_tokens_count'].idxmin()
print(df.loc[ind]['issue_body'])

Stephen requested


In [18]:
# Remove outliers with small issue description
print("Too small issue description: {} rows to be deleted".format(
    len(df[df['issue_tokens_count'] < 13]))
)

Too small issue description: 84 rows to be deleted


In [19]:
# Remove none values in issue_tokens_count
print("Too small issue description: {} rows to be deleted".format(
    len(df[df['issue_tokens_count'].isnull()]))
)

Too small issue description: 1 rows to be deleted


# Outliers filters
We consider 1-st and 99-th quantiles are acceptable for outliers filtration

In [32]:
initial = 4351064
len(df)

10195

In [33]:
changed_files_count_filter = len(df[df['changed_files_count'] <= 22])
changed_lines_count_filter = len(df[df['changed_lines_count'] <= 594])
issue_tokens_count_filter_lo = len(df[df['issue_tokens_count'] >= 13])
issue_tokens_count_filter_up = len(df[df['issue_tokens_count'] <= 4491])
tokens_filter = len(df.dropna())


print(len(df) - changed_files_count_filter, (len(df) - changed_files_count_filter) / initial * 100)
print(len(df) - changed_lines_count_filter, (len(df) - changed_lines_count_filter) / initial * 100)
print(len(df) - tokens_filter, (len(df) - tokens_filter) / initial * 100)
print(len(df) - issue_tokens_count_filter_lo, (len(df) - issue_tokens_count_filter_lo) / initial * 100)
print(len(df) - issue_tokens_count_filter_up, (len(df) - issue_tokens_count_filter_up) / initial * 100)

100 0.0022982884186488637
102 0.002344254187021841
43 0.0009882640200190116
85 0.0019535451558515345
103 0.00236723707120833


In [34]:
# Delete all none values
df.dropna(inplace=True)
print("{} rows left".format(len(df)))

10152 rows left


In [35]:
# Delete diff outliers
df = df[df['changed_files_count'] <= 22]
df = df[df['changed_lines_count'] <= 594]
print("{} rows left".format(len(df)))

9983 rows left


In [36]:
# Delete issue outliers
df = df[df['issue_tokens_count'] >= 13]
df = df[df['issue_tokens_count'] <= 4491]
print("{} rows left".format(len(df)))

9801 rows left
